# Neo4j Data prep

- Algorithm for movie data mining

In [2]:
import pandas as pd
import numpy as np

In [3]:
# votes_df = pd.read_pickle('NebflixPickles/votes_df.pkl')

sim_df = pd.read_pickle('/Users/benmurphy/OneDrive/Projects/Nebflix/DataProcessing/Pickles/cos_sim_df.pkl')

cluster_df = pd.read_pickle('/Users/benmurphy/OneDrive/Projects/Nebflix/DataProcessing/Pickles/clusterPrecomputed_df.pkl')

ratings = pd.read_csv('/Users/benmurphy/OneDrive/Projects/Nebflix/DataProcessing/MovieLensFilteredData/ratingsFiltered.csv')



In [3]:
cluster_df.head()

,cluster,movies,cluster_size
53,96,"[168, 1275, 2947, 2948, 2949, 2991, 2993, 3635...",11
40,154,"[110, 1225, 1299, 1416, 1960, 2336, 30749, 314...",11
57,186,"[175, 193, 926, 1103, 1124, 1172, 3083, 3160, ...",11
72,156,"[232, 308, 902, 909, 910, 916, 1235, 2324, 529...",10
68,41,"[215, 222, 265, 306, 4014, 5015, 8533, 8638, 6...",9


In [33]:
def get_similarity_matrix_of_cluster(cluster_number, cluster_df, similarity_df):
    temp = cluster_df.loc[cluster_df['cluster'] == cluster_number] # get temporary dataframe with one row, our cluster
    
    my_query = temp.movies.iloc[0].copy() # get movies in cluster
    my_query.append('movieId') # apend title for querying
    
    temp_similarity_df = similarity_df[my_query]
    return np.array(temp_similarity_df.loc[temp_similarity_df.movieId.isin(my_query)].drop('movieId', axis=1)), my_query.copy()

# Create csv file with all relationships within clusters

In [34]:
df_list = []

for i in cluster_df.cluster:
    my_matrix, my_movies = get_similarity_matrix_of_cluster(cluster_number=i, cluster_df=cluster_df, similarity_df = sim_df)
    my_movies.remove('movieId')
    
    to_match_from = 1

    for i,x in enumerate(my_movies):
        for j,y in enumerate(my_movies[to_match_from:]):
            
#             print(x,y,round(my_matrix[i][j+to_match_from], 4))
            
            df_list.append({'MovieIdFrom': x,
                            'MovieIdTo': y,
                            'Value': round(my_matrix[i][j+to_match_from], 4)})
        to_match_from+=1


relationship_df = pd.DataFrame(df_list)
relationship_df.head(10)

,MovieIdFrom,MovieIdTo,Value
0,168,1275,0.7611
1,168,2947,0.7446
2,168,2948,0.7369
3,168,2949,0.7442
4,168,2991,0.6207
5,168,2993,0.7019
6,168,3635,0.5396
7,168,3638,0.5556
8,168,3984,0.6949
9,168,6541,0.7604


In [35]:
relationship_df.to_csv('clusterItemRelationships.csv', index=False)

# Create csv file with items in each cluster

In [36]:
cluster_df.head(1)

,cluster,movies,cluster_size,avg_ratings
53,96,"[168, 1275, 2947, 2948, 2949, 2991, 2993, 3635...",11,4805.818182


In [37]:
df_list = []

for i,v in cluster_df.iterrows():

    for j in v.movies:
        
        df_list.append({'clusterNumber': v.cluster,
                        'movieId': j } )

    
clusterItems_df = pd.DataFrame(df_list)
clusterItems_df.head()

,clusterNumber,movieId
0,96,168
1,96,1275
2,96,2947
3,96,2948
4,96,2949


In [38]:
clusterItems_df.to_csv('clusterItems.csv', index=False)

# Create csv file for cluster numbers and average number of ratings per cluster

In [23]:
cluster_df['avg_ratings'] = 0
cluster_df.head(5)

,cluster,movies,cluster_size,avg_ratings
53,96,"[168, 1275, 2947, 2948, 2949, 2991, 2993, 3635...",11,0
40,154,"[110, 1225, 1299, 1416, 1960, 2336, 30749, 314...",11,0
57,186,"[175, 193, 926, 1103, 1124, 1172, 3083, 3160, ...",11,0
72,156,"[232, 308, 902, 909, 910, 916, 1235, 2324, 529...",10,0
68,41,"[215, 222, 265, 306, 4014, 5015, 8533, 8638, 6...",9,0


In [29]:
for i,v in cluster_df.iterrows():
    
    cluster_item_views = []
   
    for j in v.movies:
        
        movie_ratings = ratings.loc[ratings['movieId']==j].shape[0]
        cluster_item_views.append(movie_ratings)
        
    cluster_item_views = np.array(cluster_item_views)
    
    cluster_df.loc[cluster_df['cluster']==v.cluster, 'avg_ratings'] = cluster_item_views.mean()

cluster_df.head()

,cluster,movies,cluster_size,avg_ratings
53,96,"[168, 1275, 2947, 2948, 2949, 2991, 2993, 3635...",11,4805.818182
40,154,"[110, 1225, 1299, 1416, 1960, 2336, 30749, 314...",11,5901.727273
57,186,"[175, 193, 926, 1103, 1124, 1172, 3083, 3160, ...",11,4025.090909
72,156,"[232, 308, 902, 909, 910, 916, 1235, 2324, 529...",10,4765.300000
68,41,"[215, 222, 265, 306, 4014, 5015, 8533, 8638, 6...",9,4128.888889


In [30]:
df_list = []

for i,v in cluster_df.iterrows():
        
    df_list.append({'clusterNumber': v.cluster,
                    'avg_ratings': v.avg_ratings})

    
eachCluster_df = pd.DataFrame(df_list)
eachCluster_df.head()

,avg_ratings,clusterNumber
0,4805.818182,96
1,5901.727273,154
2,4025.090909,186
3,4765.300000,156
4,4128.888889,41


In [31]:
eachCluster_df.to_csv('clusterNumber.csv', index=False)

# *********************************